# Simplex algorithm

assumes LP problem to be in canonical form 

Max/Min $Z = c^{T}X$ \
s.t $AX \leq b$ \
    $X \geq 0$ 
    
  

In [2]:
import numpy as np

def minimizer_simplex(A,b,c):
    #minimization problem
    # A, b, c as per the standard form given 
    n_eq = A.shape[0]
    n_unk = c.shape[0] + n_eq #original variables + slack variables
    n_slack = n_eq #no. of slack variables
    #construct initial tableau
    coef_slack_z = np.zeros(n_slack, float)  #initially the coeff of slack variables in z
    coef_in_z = np.hstack((c, coef_slack_z))  #coeff of basic + non-basic variables in z

    coeff_basic = np.zeros((n_slack, 2))

    for i in range(coeff_basic.shape[0]):
        coeff_basic[i][0] = c.shape[0]+1+i
        coeff_basic[i][1] = 0

    ## A and an identity matrix of size n_slack * n_slack

    slack_table = np.identity(n_slack, float)

    tableau = np.hstack((A,slack_table))

    finished = False
    unbounded_flag = False
    itr = 0
    print('Minimization Simplex')
    print('=====================')

    while not finished:
        itr += 1
        print('iteration: {}'.format(itr))
        print('+++++++++++++++++++')
        print('tableau:')
        print(tableau)
        print('\n')
        #need to calculate dot product of coeff of basic and each column of tableau

        zj = []
        for column in tableau.T:
                zj.append(np.dot(coeff_basic[:,1], column))

        print('zj :{}'.format(zj))

        #find cj-zj : the relative profits for this round


        cj_zj = [c-z for (c,z) in zip(coef_in_z, zj)]
        print('cj-zj : {}'.format(cj_zj))
        min_profit = min(cj_zj)
        print('min profit for this round: {}'.format(min_profit))
        if min_profit < 0:
            #LP is not finished yet
            entering = np.argmin(cj_zj)
            b_i = np.array([a/b if b!=0 else np.inf for (a,b) in zip (b, tableau[:,entering])], float)
            print('b_i : {}'.format(b_i))
            min_ratio = min(b_i)
            if(min_ratio > 0):
                #here the LP is not unbounded
                leaving = np.argmin(b_i)
                print('entering: {}, leaving: {}'.format(entering,leaving))
                pivot = tableau[leaving][entering]
                print('pivot: {}'.format(pivot))

                #divide the line of pivot using the pivot
                #this is the leaving row. 

                next_round_tableau = np.zeros(tableau.shape)

                leaving_var = c.shape[0] + leaving + 1
                print('leaving var : {}'.format(leaving_var))

                # since pivot column variable is entering, change the coefficient and index in coeff_basic_var
                
                coeff_basic[leaving][1] = coef_in_z[entering]
                coeff_basic[leaving][0] = entering+1
                print('coeff_basic_var after updating: {}'.format(coeff_basic))
                
                #to construct tableau for next round
                #dividing pivot row with pivot
                next_round_tableau[leaving] = tableau[leaving]/pivot

                #for each index, the tranformation is done to obtain new value
                row = 0
                while row != n_eq:
                    if row != leaving:
                        for col in range(tableau.shape[1]):
                            if col != entering:
                                next_round_tableau[row][col] = (tableau[row][col]-((tableau[row][entering]*tableau[leaving][col])/pivot))
                    row += 1


                #finally, the basic variables will form identity matrix. assigning 1 in those indices
                for i in range(next_round_tableau.shape[0]):
                    next_round_tableau[i][i+c.shape[0]] = 1
                tableau = next_round_tableau
                print('\n\n')
            else:
                #unbounded LP case
                unbounded_flag = True
                print('LP unbounded!')
                finished = True
        else:
            finished = True
            if not unbounded_flag:
                print('\n\nSimplex finished\n')
                print('optimal values \(x values index and slack variables\): \n {}'.format(coeff_basic))
            #optimum = np.dot(c,coeff_basic[:,1])

def simplex_solver(A, b, c, MIN=0):
    if MIN:
        minimizer_simplex(A,b,c)
    else:
        # A, b, c as per the standard form given 
        n_eq = A.shape[0]
        n_unk = c.shape[0] + n_eq #original variables + slack variables
        n_slack = n_eq #no. of slack variables
        #construct initial tableau
        coef_slack_z = np.zeros(n_slack, float)  #initially the coeff of slack variables in z
        coef_in_z = np.hstack((c, coef_slack_z))  #coeff of basic + non-basic variables in z

        coeff_basic = np.zeros((n_slack, 2), float)

        for i in range(coeff_basic.shape[0]):
            coeff_basic[i][0] = c.shape[0]+1+i
            coeff_basic[i][1] = 0

        ## A and an identity matrix of size n_slack * n_slack

        slack_table = np.identity(n_slack, float)

        tableau = np.hstack((A,slack_table))

        finished = False
        unbounded_flag = False
        itr = 0
        print('Maximization Simplex')
        print('====================')

        while not finished:
            itr += 1
            print('iteration: {}'.format(itr))
            print('+++++++++++++++++++')
            print('tableau:')
            print(tableau)
            print('\n')
            #need to calculate dot product of coeff of basic and each column of tableau

            zj = []
            for column in tableau.T:
                    zj.append(np.dot(coeff_basic[:,1], column))

            print('zj :{}'.format(zj))

            #find cj-zj : the relative profits for this round


            cj_zj = [c-z for (c,z) in zip(coef_in_z, zj)]
            print('cj-zj : {}'.format(cj_zj))
            max_profit = max(cj_zj)
            print('max profit for this round: {}'.format(max_profit))
            if max_profit > 0:
                #LP is not finished yet
                entering = np.argmax(cj_zj)
                b_i = np.array([a/b if b!=0 else np.inf for (a,b) in zip (b, tableau[:,entering])], float)
                print('b_i : {}'.format(b_i))
                min_ratio = min(b_i)
                if(min_ratio > 0):
                    #here the LP is not unbounded
                    leaving = np.argmin(b_i)
                    print('entering: {}, leaving: {}'.format(entering,leaving))
                    pivot = tableau[leaving][entering]
                    print('pivot: {}'.format(pivot))

                    #divide the line of pivot using the pivot
                    #this is the leaving row. 

                    next_round_tableau = np.zeros(tableau.shape, float)

                    leaving_var = c.shape[0] + leaving + 1
                    print('leaving var : {}'.format(leaving_var))

                    # since pivot column variable is entering, change the coefficient and index in coeff_basic_var

                    coeff_basic[leaving][1] = coef_in_z[entering]
                    coeff_basic[leaving][0] = entering+1
                    print('coeff_basic_var after updating: {}'.format(coeff_basic))

                    #to construct tableau for next round
                    #dividing pivot row with pivot
                    next_round_tableau[leaving] = tableau[leaving]/pivot

                    #for each index, the tranformation is done to obtain new value
                    row = 0
                    while row != n_eq:
                        if row != leaving:
                            for col in range(tableau.shape[1]):
                                if col != entering:
                                    next_round_tableau[row][col] = (tableau[row][col]-((tableau[row][entering]*tableau[leaving][col])/pivot))
                        row += 1


                    #finally, the basic variables will form identity matrix. assigning 1 in those indices
                    for i in range(next_round_tableau.shape[0]):
                        next_round_tableau[i][i+c.shape[0]] = 1
                    tableau = next_round_tableau
                    print('\n\n')
                else:
                    #unbounded LP case
                    unbounded_flag = True
                    print('LP unbounded!')
                    finished = True
            else:
                finished = True
                if not unbounded_flag:
                    print('\n\nSimplex finished\n')
                    print('optimal values \(x values index and slack variables\): \n {}'.format(coeff_basic))
                #optimum = np.dot(c,coeff_basic[:,1])


#driver
if __name__ == '__main__':
    A = np.array([[1, 1], [0, 1], [1, 2]])
    b = np.array([6, 3, 9])
    c = np.array([2, 5])
    simplex_solver(A,b,c)
    
#     A = np.array(([2, 1],
#                   [1, 1]), float)
#     b = np.array([3, 2], float)
#     c = np.array([6, 4], float)
#     simplex_solver(A,b,c)

Maximization Simplex
iteration: 1
+++++++++++++++++++
tableau:
[[1. 1. 1. 0. 0.]
 [0. 1. 0. 1. 0.]
 [1. 2. 0. 0. 1.]]


zj :[0.0, 0.0, 0.0, 0.0, 0.0]
cj-zj : [2.0, 5.0, 0.0, 0.0, 0.0]
max profit for this round: 5.0
b_i : [6.  3.  4.5]
entering: 1, leaving: 1
pivot: 1.0
leaving var : 4
coeff_basic_var after updating: [[3. 0.]
 [2. 5.]
 [5. 0.]]



iteration: 2
+++++++++++++++++++
tableau:
[[ 1.  0.  1. -1.  0.]
 [ 0.  1.  0.  1.  0.]
 [ 1.  0.  0. -2.  1.]]


zj :[0.0, 5.0, 0.0, 5.0, 0.0]
cj-zj : [2.0, 0.0, 0.0, -5.0, 0.0]
max profit for this round: 2.0
b_i : [ 6. inf  9.]
entering: 0, leaving: 0
pivot: 1.0
leaving var : 3
coeff_basic_var after updating: [[1. 2.]
 [2. 5.]
 [5. 0.]]



iteration: 3
+++++++++++++++++++
tableau:
[[ 1.  0.  1. -1.  0.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  0. -1. -1.  1.]]


zj :[2.0, 5.0, 2.0, 3.0, 0.0]
cj-zj : [0.0, 0.0, -2.0, -3.0, 0.0]
max profit for this round: 0.0


Simplex finished

optimal values \(x values index and slack variables\): 
 [[1. 2.]
 [2. 5.]
 